In [1]:
# want to preprocess text
# train against some dataset
# evaluate w/ rouge score
# test the summarizer
# upload to hugging-face


In [2]:
pip install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install transformers datasets evaluate rouge_score

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from huggingface_hub import login
login()

In [2]:
import os

# Existing PATH
current_path = os.environ['PATH']

# Paths to add
git_cmd_path = r"C:\Program Files\Git\cmd"
git_bin_path = r"C:\Program Files\Git\bin"

# Add them if not already present
paths_to_add = [git_cmd_path, git_bin_path]
for path in paths_to_add:
    if path not in current_path:
        current_path = path + ";" + current_path  # Prepend to prioritize
        print(f"Added: {path}")

# Update the PATH in the environment
os.environ['PATH'] = current_path


Added: C:\Program Files\Git\cmd
Added: C:\Program Files\Git\bin


In [3]:
os.environ['PATH']

'C:\\Program Files\\Git\\bin;C:\\Program Files\\Git\\cmd;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0\\;C:\\WINDOWS\\System32\\OpenSSH\\;C:\\Program Files (x86)\\NVIDIA Corporation\\PhysX\\Common;C:\\Program Files\\dotnet\\;C:\\Program Files\\NVIDIA Corporation\\NVIDIA app\\NvDLISR;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\;C:\\Program Files\\NVIDIA Corporation\\NVIDIA NvDLISR;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0\\;C:\\WINDOWS\\System32\\OpenSSH\\;C:\\Program Files (x86)\\NVIDIA Corporation\\PhysX\\Common;C:\\Program Files\\dotnet\\;C:\\Users\\Naja-Lee\\AppData\\Local\\Programs\\Python\\Launcher\\;C:\\Users\\Naja-Lee\\AppData\\Local\\Microsoft\\WindowsApps;C:\\Users\\Naja-Lee\

In [4]:
!git clone https://github.com/najalee/nlp-project.git

fatal: destination path 'nlp-project' already exists and is not an empty directory.


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from datasets import load_dataset

ds = load_dataset("abisee/cnn_dailymail", "3.0.0")

In [7]:
ds = ds["train"].train_test_split(test_size=0.2)

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 229690
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 57423
    })
})

In [9]:
# now we preprocess the loaded datatset
from transformers import AutoTokenizer

In [10]:
# load the google t5 tokenizer

loadedTokenizer = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(loadedTokenizer)

In [11]:
# tell T5 that this is a summzarization task
prefix = "summarize: "

def preprocess_function(artSum):
    inputs = [prefix + doc for doc in artSum["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)


    labels = tokenizer(text_target=artSum["highlights"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
tokenized_ds = {}
tokenized_ds["train"] = ds["train"].shuffle(seed=42).select(range(20000)).map(preprocess_function, batched=True) # create a small dataset so training doesnt take 500 years
tokenized_ds["test"] = ds["test"].shuffle(seed=42).select(range(1000)).map(preprocess_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [123]:
# babyds

In [124]:
# tokenized_ds = babyds.map(preprocess_function, batched=True)
# tokenized_ds = babyds["test"].map(preprocess_function, batched=True)

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=loadedTokenizer)

In [15]:
# evaluate w/ rouge score
import evaluate

rouge = evaluate.load("rouge")

In [16]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}
    

In [17]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(loadedTokenizer)

In [18]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))



True
NVIDIA GeForce RTX 2070


In [19]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # Example: move tokenized inputs to device
# tokenized_ds = tokenized_ds.to(device)


In [25]:
training_args = Seq2SeqTrainingArguments(
    output_dir="sum-it_model",
    eval_strategy="no",
    learning_rate=2e-5,
    #per_device_train_batch_size=16,
    #per_device_eval_batch_size=16,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=False,
    fp16=True, #change to bf16=True for XPU
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# training_args = Seq2SeqTrainingArguments(
#     output_dir="my_awesome_billsum_model",
#     #eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=4,
#     predict_with_generate=True,
#     fp16=True, #change to bf16=True for XPU
#     push_to_hub=False,
#     #dataloader_num_workers=4,  # or 8, depending on your CPU
# )

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_ds["train"],
#     eval_dataset=tokenized_ds["test"],
#     processing_class=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# trainer.train()

Step,Training Loss
500,1.849400
1000,1.905700


TrainOutput(global_step=1250, training_loss=1.8828989990234375, metrics={'train_runtime': 868.383, 'train_samples_per_second': 23.031, 'train_steps_per_second': 1.439, 'total_flos': 5407416729993216.0, 'train_loss': 1.8828989990234375, 'epoch': 1.0})

In [26]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/najalee/sum-it_model/commit/ee482d315ceb45b0969bdc243da5e8d9a05c9a18', commit_message='End of training', commit_description='', oid='ee482d315ceb45b0969bdc243da5e8d9a05c9a18', pr_url=None, repo_url=RepoUrl('https://huggingface.co/najalee/sum-it_model', endpoint='https://huggingface.co', repo_type='model', repo_id='najalee/sum-it_model'), pr_revision=None, pr_num=None)

In [29]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
text = "summarize: hello everybody i hope we're having a wonderful night first i wanted to discuss how valorant works! it is a 5v5 first person shooter, and the goal is to either plant the bomb or defuse it, depending on if youre on the attacking or defending side, respectivley. my favorite agent (that's what you call the characters) is raze! she is a duelist, meaning she's meant to be one of the first in to get the first pick (kill), getting her team onto site. she has a bomb, two satchels that explode and launch her in the air, a boombot that tracks people and blows up (i call it a roombah), and a bazooka as her ultimate ability! i havent played in a while so i'm probably not that good at her anymore."

from transformers import pipeline

summarizer = pipeline("summarization", 
                      model="najalee/sum-it_model", 
                      min_length=20)
summarizer(text)

Device set to use cuda:0


[{'summary_text': "the goal is to either plant the bomb or defuse it, depending on if youre on the attacking or defending side, respectivley . she's meant to be one of the first in to get the first pick (kill), getting her team onto site ."}]

In [61]:
text = "hello everybody i hope we're having a wonderful night first i wanted to discuss how valorant works! it is a 5v5 first person shooter, and the goal is to either plant the bomb or defuse it, depending on if youre on the attacking or defending side, respectivley. my favorite agent (that's what you call the characters) is raze! she is a duelist, meaning she's meant to be one of the first in to get the first pick (kill), getting her team onto site. she has a bomb, two satchels that explode and launch her in the air, a boombot that tracks people and blows up (i call it a roombah), and a bazooka as her ultimate ability! i havent played in a while so i'm probably not that good at her anymore."

In [62]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("najalee/my_awesome_billsum_model")
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=600).input_ids

In [63]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("najalee/my_awesome_billsum_model")
outputs = model.generate(inputs, max_new_tokens=100, min_length=50, do_sample=False)

In [64]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

"everyone i hope we're having a wonderful night first i wanted to discuss how valorant works! it is a 5v5 first person shooter, and the goal is to either plant the bomb or defuse it, depending on if youre on the attacking or defending side, respectivley. she's meant to be one of the first in to get the first pick (kill), getting her team onto site. she has a bomb,"